<h1>Download required libraries and git clone tensorflow/models</h1>

In [0]:
#Use tensorflowGPU 1.12.0 because TensorflowJS won't work with newer versions (unsupported ops)
!pip install tensorflow-gpu==1.12.0

#Download Protobuf
!apt-get install protobuf-compiler python-pil python-lxml python-tk

#Colab has CUDA 10.1 installed which isn't compatible with TF-gpu 1.12, so install CUDA 9.0 (It'll remove 10.1 in the process)
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

!git clone https://github.com/Thorben-b/models.git

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [0]:
#This completes the installation the Object Detection API
%cd /content/models/research
!python setup.py build
!python setup.py install

In [0]:
#Compile the Protobuf libraries and test the installation of the Object Detection API
!protoc object_detection/protos/*.proto --python_out=.
!python object_detection/builders/model_builder_test.py

In [0]:
#Convert the json files (with the bounding box coordinates) to csv
%cd object_detection/
!python json_to_csv.py

In [0]:
#Generate TFrecords from the csv files
!python3 generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train/ --output_path=train.record
!python3 generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test/ --output_path=test.record

In [0]:
%cd ..

**Training the actual model**

In [0]:
#Train the model using train.py script from TensorFlow Object Detection API
#--train_dir sets the model which'll be used
#--pipeline_config_path is the path to the config file

!python3 object_detection/legacy/train.py --logtostderr --train_dir=object_detection/ssd_model_v1/ \
--pipeline_config_path=object_detection/training/pipeline.config

**Another train script, this one gives a bit more feedback during training, but the above script was used during the initial training of the model and is used in the application**

In [0]:
!python object_detection/model_main.py --logtostderr --model_dir=object_detection/ssd_v3_small --pipeline_config_path=object_detection/training/pipeline.config

**Save the model to /object_detection/kenteken_graph**

In [0]:
!python3 object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path object_detection/training/pipeline.config \
    --trained_checkpoint_prefix object_detection/ssd_v3_small/model.ckpt-8456 \
    --output_directory object_detection/kenteken_graph_v1

In [0]:
#Zip the model (not necessary)
!zip -r object_detection/trained_model_v3_8456.zip object_detection/kenteken__v3_graph_8456

**Convert the model to TensorFlow JS**

In [0]:
#First install TensorFlowJS
!pip install tensorflowjs

#Show the converter where the input path is (it'll look for the savedmodel file)
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --saved_model_tags=serve object_detection/kenteken_graph_v1/saved_model object_detection/kenteken_graph_v1/TFJS